In [12]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from ipywidgets import interact, widgets
from scipy.stats import multivariate_normal

# Set up the Case 1 parameters
class Case1DecisionBoundary:
    def __init__(self):
        # Class parameters
        self.mu1 = np.array([2, 2])      # Mean of class 1
        self.mu2 = np.array([-1, -1])    # Mean of class 2  
        self.sigma_squared = 1.0         # Common variance σ²
        self.Sigma = self.sigma_squared * np.eye(2)  # Covariance matrix σ²I
        
        # Create coordinate grids
        self.x1 = np.linspace(-6, 6, 150)
        self.x2 = np.linspace(-6, 6, 150)
        self.X1, self.X2 = np.meshgrid(self.x1, self.x2)
        
    def discriminant_functions(self, prior_class1):
        """Calculate linear discriminant functions for Case 1"""
        prior_class2 = 1 - prior_class1
        
        # For Case 1: g_i(x) = w_i^T * x + w_i0
        # where w_i = Σ^(-1) * μ_i
        # and w_i0 = -0.5 * μ_i^T * Σ^(-1) * μ_i + ln(π_i)
        
        Sigma_inv = np.linalg.inv(self.Sigma)
        
        # Weight vectors
        w1 = Sigma_inv @ self.mu1
        w2 = Sigma_inv @ self.mu2
        
        # Bias terms
        w10 = -0.5 * self.mu1.T @ Sigma_inv @ self.mu1 + np.log(prior_class1)
        w20 = -0.5 * self.mu2.T @ Sigma_inv @ self.mu2 + np.log(prior_class2)
        
        return w1, w10, w2, w20
    
    def calculate_decision_regions(self, prior_class1):
        """Calculate discriminant values and decision regions"""
        w1, w10, w2, w20 = self.discriminant_functions(prior_class1)
        
        # Calculate g1(x) and g2(x) on the grid
        g1 = w1[0] * self.X1 + w1[1] * self.X2 + w10
        g2 = w2[0] * self.X1 + w2[1] * self.X2 + w20
        
        # Decision function: g1(x) - g2(x)
        decision = g1 - g2
        
        return g1, g2, decision, w1, w10, w2, w20

# Create the visualization instance
boundary_viz = Case1DecisionBoundary()

def create_visualization(prior_class1=0.5):
    """Create comprehensive visualization of Case 1 decision boundary"""
    
    # Calculate discriminant functions and decision regions
    g1, g2, decision, w1, w10, w2, w20 = boundary_viz.calculate_decision_regions(prior_class1)
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. 2D Decision Boundary Plot
    ax1 = axes[0, 0]
    
    # Plot decision regions with different colors
    ax1.contourf(boundary_viz.X1, boundary_viz.X2, decision, 
                levels=[0, np.inf], colors=['lightblue'], alpha=0.6)
    ax1.contourf(boundary_viz.X1, boundary_viz.X2, decision, 
                levels=[-np.inf, 0], colors=['lightcoral'], alpha=0.6)
    
    # Plot decision boundary (where g1 = g2)
    ax1.contour(boundary_viz.X1, boundary_viz.X2, decision, 
               levels=[0], colors='black', linewidths=3)
    
    # Plot class means
    ax1.plot(boundary_viz.mu1[0], boundary_viz.mu1[1], 'bo', markersize=15, 
            markeredgecolor='darkblue', markeredgewidth=2, label='μ₁ (Class 1)')
    ax1.plot(boundary_viz.mu2[0], boundary_viz.mu2[1], 'ro', markersize=15, 
            markeredgecolor='darkred', markeredgewidth=2, label='μ₂ (Class 2)')
    
    ax1.set_xlabel('x₁', fontsize=12)
    ax1.set_ylabel('x₂', fontsize=12)
    ax1.set_title(f'2D Decision Boundary\nπ₁ = {prior_class1:.3f}, π₂ = {1-prior_class1:.3f}', 
                 fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_aspect('equal')
    
    # 2. Contour plot of discriminant functions
    ax2 = axes[0, 1]
    
    contour1 = ax2.contour(boundary_viz.X1, boundary_viz.X2, g1, 
                          levels=10, colors='blue', alpha=0.6)
    contour2 = ax2.contour(boundary_viz.X1, boundary_viz.X2, g2, 
                          levels=10, colors='red', alpha=0.6)
    ax2.contour(boundary_viz.X1, boundary_viz.X2, decision, 
               levels=[0], colors='black', linewidths=3)
    
    ax2.clabel(contour1, fontsize=8, fmt='%.1f')
    ax2.clabel(contour2, fontsize=8, fmt='%.1f')
    
    ax2.plot(boundary_viz.mu1[0], boundary_viz.mu1[1], 'bo', markersize=12)
    ax2.plot(boundary_viz.mu2[0], boundary_viz.mu2[1], 'ro', markersize=12)
    
    ax2.set_xlabel('x₁')
    ax2.set_ylabel('x₂')
    ax2.set_title('Contour Lines of g₁(x) and g₂(x)', fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.set_aspect('equal')
    
    # 3. Effect of Prior on Boundary Position
    ax3 = axes[0, 2]
    
    # Show multiple boundaries for different priors
    prior_values = [0.1, 0.3, 0.5, 0.7, 0.9]
    colors = ['purple', 'orange', 'black', 'green', 'brown']
    
    for prior_val, color in zip(prior_values, colors):
        _, _, decision_temp, _, _, _, _ = boundary_viz.calculate_decision_regions(prior_val)
        ax3.contour(boundary_viz.X1, boundary_viz.X2, decision_temp, 
                   levels=[0], colors=[color], linewidths=2, 
                   linestyles='-' if prior_val == prior_class1 else '--')
    
    # Highlight current boundary
    ax3.contour(boundary_viz.X1, boundary_viz.X2, decision, 
               levels=[0], colors='red', linewidths=4)
    
    ax3.plot(boundary_viz.mu1[0], boundary_viz.mu1[1], 'bo', markersize=12)
    ax3.plot(boundary_viz.mu2[0], boundary_viz.mu2[1], 'ro', markersize=12)
    
    ax3.set_xlabel('x₁')
    ax3.set_ylabel('x₂')
    ax3.set_title('Effect of Prior Probability\n(Thick red = current)', fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.set_aspect('equal')
    
    # Add legend for different priors
    legend_elements = [plt.Line2D([0], [0], color=color, linewidth=2, 
                                 label=f'π₁={prior_val}') 
                      for prior_val, color in zip(prior_values, colors)]
    ax3.legend(handles=legend_elements, fontsize=9, loc='upper right')
    
    # 4. Mathematical Details (Text)
    ax4 = axes[1, 0]
    ax4.axis('off')
    
    # Calculate decision boundary line equation
    w_diff = w1 - w2
    w0_diff = w10 - w20
    
    math_text = f"""CASE 1: Equal Covariance Matrices (Σᵢ = σ²I)

Given Parameters:
• μ₁ = [{boundary_viz.mu1[0]}, {boundary_viz.mu1[1]}]
• μ₂ = [{boundary_viz.mu2[0]}, {boundary_viz.mu2[1]}]
• σ² = {boundary_viz.sigma_squared}
• π₁ = {prior_class1:.3f}, π₂ = {1-prior_class1:.3f}

Linear Discriminant Functions:
gᵢ(x) = wᵢᵀx + wᵢ₀

Calculated Weights:
• w₁ = [{w1[0]:.2f}, {w1[1]:.2f}]
• w₁₀ = {w10:.3f}
• w₂ = [{w2[0]:.2f}, {w2[1]:.2f}]
• w₂₀ = {w20:.3f}

Decision Boundary:
{w_diff[0]:.3f}x₁ + {w_diff[1]:.3f}x₂ + {w0_diff:.3f} = 0

Key Point: The boundary is LINEAR because
covariance matrices are equal!"""
    
    ax4.text(0.05, 0.95, math_text, transform=ax4.transAxes, fontsize=11,
            verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    # 5. Likelihood surfaces using scipy.stats.multivariate_normal
    ax5 = axes[1, 1]
    
    # Create meshgrid for likelihood calculation
    pos = np.dstack((boundary_viz.X1, boundary_viz.X2))
    
    # Calculate multivariate normal distributions
    rv1 = multivariate_normal(boundary_viz.mu1, boundary_viz.Sigma)
    rv2 = multivariate_normal(boundary_viz.mu2, boundary_viz.Sigma)
    
    likelihood1 = rv1.pdf(pos)
    likelihood2 = rv2.pdf(pos)
    
    # Apply priors
    posterior1 = prior_class1 * likelihood1
    posterior2 = (1 - prior_class1) * likelihood2
    
    # Plot contours of posterior probabilities
    contour_post1 = ax5.contour(boundary_viz.X1, boundary_viz.X2, posterior1, 
                               levels=8, colors='blue', alpha=0.8)
    contour_post2 = ax5.contour(boundary_viz.X1, boundary_viz.X2, posterior2, 
                               levels=8, colors='red', alpha=0.8)
    
    # Plot decision boundary
    ax5.contour(boundary_viz.X1, boundary_viz.X2, decision, 
               levels=[0], colors='black', linewidths=3)
    
    ax5.plot(boundary_viz.mu1[0], boundary_viz.mu1[1], 'bo', markersize=12)
    ax5.plot(boundary_viz.mu2[0], boundary_viz.mu2[1], 'ro', markersize=12)
    
    ax5.set_xlabel('x₁')
    ax5.set_ylabel('x₂')
    ax5.set_title('Posterior Probability Contours\nπᵢp(x|ωᵢ)', fontweight='bold')
    ax5.grid(True, alpha=0.3)
    ax5.set_aspect('equal')
    
    # 6. 1D Cross-section along line connecting means
    ax6 = axes[1, 2]
    
    # Create line from mu2 to mu1
    t = np.linspace(0, 1, 100)
    line_points = np.outer(1-t, boundary_viz.mu2) + np.outer(t, boundary_viz.mu1)
    
    # Calculate discriminant values along this line
    g1_line = []
    g2_line = []
    for point in line_points:
        g1_val = w1[0] * point[0] + w1[1] * point[1] + w10
        g2_val = w2[0] * point[0] + w2[1] * point[1] + w20
        g1_line.append(g1_val)
        g2_line.append(g2_val)
    
    g1_line = np.array(g1_line)
    g2_line = np.array(g2_line)
    
    # Plot discriminant functions along the line
    ax6.plot(t, g1_line, 'b-', linewidth=3, label='g₁(x)')
    ax6.plot(t, g2_line, 'r-', linewidth=3, label='g₂(x)')
    ax6.axhline(y=0, color='k', linestyle='--', alpha=0.5)
    
    # Find and mark intersection point
    intersection_idx = np.argmin(np.abs(g1_line - g2_line))
    ax6.plot(t[intersection_idx], g1_line[intersection_idx], 'go', 
            markersize=12, label=f'Decision point (t={t[intersection_idx]:.3f})')
    
    ax6.set_xlabel('t (0=μ₂, 1=μ₁)')
    ax6.set_ylabel('g(x)')
    ax6.set_title('1D Cross-section\nAlong μ₂ → μ₁', fontweight='bold')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print current mathematical state
    print("="*80)
    print(f"CURRENT STATE: π₁ = {prior_class1:.3f}")
    print("="*80)
    print(f"Decision Boundary Equation: {w_diff[0]:.3f}x₁ + {w_diff[1]:.3f}x₂ + {w0_diff:.3f} = 0")
    print(f"Boundary shifts towards Class {'2' if prior_class1 < 0.5 else '1'} region")
    print("="*80)

def simple_comparison():
    """Simple comparison of different prior values"""
    print("🎯 Comparison of Different Prior Values:")
    print("="*60)
    
    prior_values = [0.1, 0.3, 0.5, 0.7, 0.9]
    
    fig, axes = plt.subplots(1, len(prior_values), figsize=(20, 4))
    
    for i, prior in enumerate(prior_values):
        ax = axes[i]
        g1, g2, decision, w1, w10, w2, w20 = boundary_viz.calculate_decision_regions(prior)
        
        # Plot decision regions
        ax.contourf(boundary_viz.X1, boundary_viz.X2, decision, 
                   levels=[0, np.inf], colors=['lightblue'], alpha=0.6)
        ax.contourf(boundary_viz.X1, boundary_viz.X2, decision, 
                   levels=[-np.inf, 0], colors=['lightcoral'], alpha=0.6)
        
        # Plot decision boundary
        ax.contour(boundary_viz.X1, boundary_viz.X2, decision, 
                  levels=[0], colors='black', linewidths=3)
        
        # Plot class means
        ax.plot(boundary_viz.mu1[0], boundary_viz.mu1[1], 'bo', markersize=12)
        ax.plot(boundary_viz.mu2[0], boundary_viz.mu2[1], 'ro', markersize=12)
        
        ax.set_xlabel('x₁')
        ax.set_ylabel('x₂')
        ax.set_title(f'π₁ = {prior}')
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()
    
    print("\n🔑 Observation: Notice how the LINEAR boundary shifts as π₁ changes!")
    print("This is the key characteristic of Case 1 (Equal Covariance Matrices)")

# Interactive widget function
def interactive_visualization():
    """Create interactive visualization using ipywidgets"""
    return interact(
        create_visualization,
        prior_class1=widgets.FloatSlider(
            value=0.5,
            min=0.01,
            max=0.99,
            step=0.01,
            description='Prior π₁:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )
    )

# Usage examples and instructions
print("🎯 DECISION BOUNDARY VISUALIZATION - CASE 1")
print("="*60)
print("\nAvailable functions:")
print("1. interactive_visualization()    # Interactive widget with slider")
print("2. create_visualization(0.7)      # Single plot with specific prior")
print("3. simple_comparison()            # Static comparison of different priors")

print("\n🔬 Mathematical Foundation:")
print("Case 1: Σᵢ = σ²I → Linear discriminant functions")
print("Decision boundary: (w₁-w₂)ᵀx + (w₁₀-w₂₀) = 0")
print("\n📊 Features:")
print("• Real-time interactive slider")
print("• Comprehensive 6-panel visualization")
print("• Mathematical details and equations")
print("• Uses scipy.stats.multivariate_normal for accuracy")

🎯 DECISION BOUNDARY VISUALIZATION - CASE 1

Available functions:
1. interactive_visualization()    # Interactive widget with slider
2. create_visualization(0.7)      # Single plot with specific prior
3. simple_comparison()            # Static comparison of different priors

🔬 Mathematical Foundation:
Case 1: Σᵢ = σ²I → Linear discriminant functions
Decision boundary: (w₁-w₂)ᵀx + (w₁₀-w₂₀) = 0

📊 Features:
• Real-time interactive slider
• Comprehensive 6-panel visualization
• Mathematical details and equations
• Uses scipy.stats.multivariate_normal for accuracy


In [13]:
interactive_visualization()

interactive(children=(FloatSlider(value=0.5, description='Prior π₁:', layout=Layout(width='500px'), max=0.99, …

<function __main__.create_visualization(prior_class1=0.5)>